In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df2015 = pd.read_csv('2015.csv')
df2016 = pd.read_csv('2016.csv')
df2017 = pd.read_csv('2017.csv')
banks = pd.read_csv('download_bank_connections.csv')
transactions = pd.read_csv('download_transactions_map.csv')

In [ ]:
#concatinating 3 dataframes
df = pd.concat([df2015,df2016,df2017])
#changing object format to datetime
df['time'] = pd.to_datetime(df['time'])
df.head()

,time,Sender,Receiver,Transaction_amount,Price,Transaction_amount_USD
0,2015-04-01,14noyXsYSBTJYsyUmyDWhSeAVbge7qi5E3,1H8rsLPgFQ3kBU7WvRGDR8X9Pv1d5k3zQF,8811.015,243.288169,2.143616e+06
1,2015-04-01,1LHXyLdpKC52988BqFXCHksU11QvXM7qRT,14noyXsYSBTJYsyUmyDWhSeAVbge7qi5E3,8972.836,243.288169,2.182985e+06
2,2015-04-01,15Bu9gyGtGrXuoLDe3erKfTj7n15mAp1Wc\n1KYJV7JJDu...,1BsrvZgK11ZAsJYokFe9pAm3z58dodQLAr,5000.000,243.288169,1.216441e+06
3,2015-04-01,1KYJV7JJDuoR4ePgqm7C4o9gidMGhRUHNH\n15X5v1G4fu...,1AXFVP7zumDPebSb516yaJyzqwQ1VQeDSX,5000.000,243.288169,1.216441e+06
4,2015-04-01,15Bu9gyGtGrXuoLDe3erKfTj7n15mAp1Wc\n1FzAohdX6E...,14aPvf4RMnktzauPcg97wQFCJRS1TMogWb,5000.000,243.288169,1.216441e+06


In [ ]:
#making sure that the format is changed
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61137 entries, 0 to 9782
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   time                    61137 non-null  datetime64[ns]
 1   Sender                  61137 non-null  object        
 2   Receiver                61137 non-null  object        
 3   Transaction_amount      61137 non-null  float64       
 4   Price                   61137 non-null  float64       
 5   Transaction_amount_USD  61137 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 3.3+ MB


In [ ]:
banks.head()

,icij_sar_id,filer_org_name_id,filer_org_name,entity_b_id,entity_b,entity_b_country,entity_b_iso_code
0,4132,standard-chartered-plc,Standard Chartered Plc,habib-metropolitan-bank-limited-karachi-pakist...,Habib Metropolitan Bank Limited,Pakistan,PAK
1,3181,standard-chartered-plc,Standard Chartered Plc,standard-chartered-bank-dubai-uae-are,Standard Chartered Bank,United Arab Emirates,ARE
2,3181,standard-chartered-plc,Standard Chartered Plc,al-rostamani-international-exchange-uae-are,Al Rostamani International Exchange,United Arab Emirates,ARE
3,3181,standard-chartered-plc,Standard Chartered Plc,kuwait-turkish-participation-bank-inc-istanbul...,Kuwait Turkish Participation Bank Inc,Turkey,TUR
4,3181,standard-chartered-plc,Standard Chartered Plc,wall-street-exchange-center-llc-dubai-uae-are,Wall Street Exchange Center Llc,United Arab Emirates,ARE


In [ ]:
transactions.head()

,id,icij_sar_id,filer_org_name_id,filer_org_name,begin_date,end_date,originator_bank_id,originator_bank,originator_bank_country,originator_iso,beneficiary_bank_id,beneficiary_bank,beneficiary_bank_country,beneficiary_iso,number_transactions,amount_transactions
0,223254,3297,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"Mar 25, 2015","Sep 25, 2015",cimb-bank-berhad,CIMB Bank Berhad,Singapore,SGP,barclays-bank-plc-london-england-gbr,Barclays Bank Plc,United Kingdom,GBR,68.0,5.689852e+07
1,223255,3297,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"Mar 30, 2015","Sep 25, 2015",cimb-bank-berhad,CIMB Bank Berhad,Singapore,SGP,barclays-bank-plc-london-england-gbr,Barclays Bank Plc,United Kingdom,GBR,118.0,1.162384e+08
2,223258,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"Jul 5, 2012","Jul 5, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden...,Skandinaviska Enskilda Banken,Sweden,SWE,NaN,5.000000e+03
3,223259,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"Jun 20, 2012","Jun 20, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden...,Skandinaviska Enskilda Banken,Sweden,SWE,NaN,9.990000e+03
4,223260,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"May 31, 2012","May 31, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden...,Skandinaviska Enskilda Banken,Sweden,SWE,NaN,1.200000e+04


In [ ]:
#getting columns names
transactions.columns

Index(['id', 'icij_sar_id', 'filer_org_name_id', 'filer_org_name',
       'begin_date', 'end_date', 'originator_bank_id', 'originator_bank',
       'originator_bank_country', 'originator_iso', 'beneficiary_bank_id',
       'beneficiary_bank', 'beneficiary_bank_country', 'beneficiary_iso',
       'number_transactions', 'amount_transactions'],
      dtype='object')

In [ ]:
#creating new filtered dataframe to speed up post-processing
trans = transactions[['begin_date', 'end_date', 'originator_bank','filer_org_name','beneficiary_bank','amount_transactions']]
trans.head()

,begin_date,end_date,originator_bank,filer_org_name,beneficiary_bank,amount_transactions
0,"Mar 25, 2015","Sep 25, 2015",CIMB Bank Berhad,The Bank of New York Mellon Corp.,Barclays Bank Plc,5.689852e+07
1,"Mar 30, 2015","Sep 25, 2015",CIMB Bank Berhad,The Bank of New York Mellon Corp.,Barclays Bank Plc,1.162384e+08
2,"Jul 5, 2012","Jul 5, 2012",Barclays Bank Plc Ho UK,The Bank of New York Mellon Corp.,Skandinaviska Enskilda Banken,5.000000e+03
3,"Jun 20, 2012","Jun 20, 2012",Barclays Bank Plc Ho UK,The Bank of New York Mellon Corp.,Skandinaviska Enskilda Banken,9.990000e+03
4,"May 31, 2012","May 31, 2012",Barclays Bank Plc Ho UK,The Bank of New York Mellon Corp.,Skandinaviska Enskilda Banken,1.200000e+04


In [ ]:
##changing object format to datetime
trans['end_date'] = pd.to_datetime(trans['end_date'])
trans['begin_date'] = pd.to_datetime(trans['begin_date'])
trans.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,begin_date,end_date,originator_bank,filer_org_name,beneficiary_bank,amount_transactions
4502,2016-08-22,2016-08-22,ING Netherland NV,Deutsche Bank AG,Rigenesis Bank AS,20415554.55
4503,2016-08-16,2016-08-16,Rigenesis Bank AS,Deutsche Bank AG,ING Netherland NV,23000000.00
4504,2016-08-16,2016-08-16,ING Netherland NV,Deutsche Bank AG,Rigenesis Bank AS,23416000.00
4505,2016-06-14,2016-06-14,Rigenesis Bank AS,Deutsche Bank AG,ING Netherland NV,33258935.96
4506,2016-06-14,2016-06-14,ING Netherland NV,Deutsche Bank AG,Rigenesis Bank AS,33259000.00


In [ ]:
#making sure that the format is changed
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4507 entries, 0 to 4506
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   begin_date           4501 non-null   datetime64[ns]
 1   end_date             4501 non-null   datetime64[ns]
 2   originator_bank      4507 non-null   object        
 3   filer_org_name       4507 non-null   object        
 4   beneficiary_bank     4507 non-null   object        
 5   amount_transactions  4507 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 211.4+ KB


In [ ]:
#selectng the columns required for analysis
df = df[['time', 'Sender', 'Transaction_amount_USD']]
df.head()

,time,Sender,Transaction_amount_USD
0,2015-04-01,14noyXsYSBTJYsyUmyDWhSeAVbge7qi5E3,2.143616e+06
1,2015-04-01,1LHXyLdpKC52988BqFXCHksU11QvXM7qRT,2.182985e+06
2,2015-04-01,15Bu9gyGtGrXuoLDe3erKfTj7n15mAp1Wc\n1KYJV7JJDu...,1.216441e+06
3,2015-04-01,1KYJV7JJDuoR4ePgqm7C4o9gidMGhRUHNH\n15X5v1G4fu...,1.216441e+06
4,2015-04-01,15Bu9gyGtGrXuoLDe3erKfTj7n15mAp1Wc\n1FzAohdX6E...,1.216441e+06


In [ ]:
result = []
#creating a loop on the df rows
for time, blockchain_transaction_usd, sender in df[['time','Transaction_amount_USD','Sender']].itertuples(index=False):
#inside creating another loop through the trans rows
  for begin_date, end_date, originator_bank, filer_org_name, beneficiary_bank, bank_transaction_usd in trans[['begin_date','end_date','originator_bank','filer_org_name','beneficiary_bank','amount_transactions']].itertuples(index=False):
#initializing the variable diff 
    diff = abs(blockchain_transaction_usd - bank_transaction_usd);
#selecting blockchain transactions made between begin_date and end_date bank transactions, and which differ from banking amount_transactions by no more than 1%
    if abs(blockchain_transaction_usd - bank_transaction_usd)/blockchain_transaction_usd <= 0.01 and time >= begin_date and time <= end_date:
#adding suspicious transactions to the result list
      result.append([time, originator_bank, filer_org_name, beneficiary_bank, bank_transaction_usd, blockchain_transaction_usd, diff, sender])

In [ ]:
#changing the numbers representation format to a more convenient one
pd.options.display.float_format ='{:,.0f}'.format
#creating the final dataframe
result = pd.DataFrame(result, columns=['time', 'originator_bank', 'filer_org_name', 'beneficiary_bank', 'bank_transaction_usd', 'blockchain_transaction_usd', 'diff_usd', 'sender'])
#looking at the result
result.head(5)

,time,originator_bank,filer_org_name,beneficiary_bank,bank_transaction_usd,blockchain_transaction_usd,diff_usd,sender
0,2015-03-31,DMS Bank & Trust Ltd.,The Bank of New York Mellon Corp.,"Hang Seng Bank, Ltd","1,026,110","1,017,569","8,541",15HgmxddgExHKTfkXwacQgKf9nx8PZEDri
1,2015-03-26,Bank of Communications,Standard Chartered Plc,Standard Chartered Bank Hong Kong Ltd,"1,251,050","1,246,427","4,623",183LytTb7GgTif1q9ggaqw2MY8GSq6SsH7
2,2015-03-26,Fifth Third Bank,"Fifth Third Bank, National Association",Fifth Third Bank,"1,245,000","1,246,427","1,427",183LytTb7GgTif1q9ggaqw2MY8GSq6SsH7
3,2015-03-26,DMS Bank & Trust Ltd.,The Bank of New York Mellon Corp.,"Hang Seng Bank, Ltd","1,026,110","1,035,335","9,225",13V8cGoAUkdRmJv1E7cbe9R7dZBt4MjQTd
4,2015-03-26,Expobank,The Bank of New York Mellon Corp.,Credit Suisse AG,"1,080,000","1,076,255","3,745",1N7YMh49QB6KkBjMfX4boriNwtcQNAzX64


In [ ]:
#converting the result to result.csv
result.to_csv('result.csv')